<a href="https://colab.research.google.com/github/azaslight/skin_cancer_classifier/blob/main/telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install python-telegram-bot --quiet
!pip install tensorflow --quiet

In [ ]:
import logging
import os
import asyncio
import numpy as np
from PIL import Image
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, CallbackContext, filters
from tensorflow.keras.models import load_model

# Вставьте ваш API Token сюда
API_TOKEN = '7789503144:AAHJxUBty6Lta5pdsrM4SVvDV8Lt3bxQNCc'

# Загрузка модели
model = load_model('/content/drive/MyDrive/skin_cancer_classifier.keras')

# Включаем логирование
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Команда /start
async def start(update: Update, context: CallbackContext):
    await update.message.reply_text('Привет! Отправь мне изображение, и я определю, есть ли признаки рака кожи.')

# Обработка изображения
async def handle_image(update: Update, context: CallbackContext):
    try:
        # Получаем последнее (самое большое) изображение из сообщения
        file = await update.message.photo[-1].get_file()

        # Логируем информацию о полученной фотографии
        logger.info(f"Получено изображение: {file.file_id}")

        # Скачиваем файл
        file_path = 'temp_image.jpg'
        await file.download_to_drive(file_path)

        # Открытие изображения и подготовка данных
        img = Image.open(file_path)
        img = img.convert('RGB')  # Убедимся, что изображение в RGB формате
        img = img.resize((64, 64))  # Изменение размера до нужного для модели
        img_array = np.array(img) / 255.0  # Нормализация
        img_array = np.expand_dims(img_array, axis=0)  # Добавляем размерность для батча

        # Логируем информацию о процессе обработки
        logger.info(f"Изображение обработано, размер: {img_array.shape}")

        # Предсказание
        prediction = model.predict(img_array)

        # Логируем предсказание
        logger.info(f"Предсказание: {prediction[0][0]}")

        # Интерпретация результата
        if prediction[0][0] >= 0.5:
            await update.message.reply_text("Это изображение может быть опасным. Рекомендуется проконсультироваться с врачом.")
        else:
            await update.message.reply_text("Этот снимок не показывает признаков рака кожи.")

        # Удаляем временный файл
        os.remove(file_path)

    except Exception as e:
        # Логируем ошибку
        logger.error(f"Произошла ошибка при обработке изображения: {str(e)}")
        await update.message.reply_text(f"Произошла ошибка при обработке изображения: {str(e)}")

# Основная функция
async def main():
    # Создаем объект приложения
    application = Application.builder().token(API_TOKEN).build()

    # Добавляем обработчики
    application.add_handler(CommandHandler('start', start))
    application.add_handler(MessageHandler(filters.PHOTO, handle_image))

    # Запуск бота
    await application.run_polling()

# Для запуска в Google Colab
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.create_task(main())
    loop.run_forever()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
